## Methods start here

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from multiprocessing import Process, Manager

def set_Data(data):
    ppmi = pd.read_csv('../../datasets/preprocessed/trans_processed_PPMI_data.csv')
    ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
    ppmi.set_index('Sentrix_position', inplace=True)
    ppmi = ppmi.transpose()

    encoder = LabelEncoder()
    label = encoder.fit_transform(ppmi['Category'])

    tr = ppmi.drop(['Category'], axis=1)
    X = tr.values
    y = label
    print(X.shape)
    print(y.shape)

    print("StratifiedSampling check")
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    split.get_n_splits(X, y)

    for train_index, test_index in split.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, data['y_test'] = y[train_index], y[test_index]

    print("Oversampling check")
    oversampler = SMOTE(random_state=42)
    X_train_sampled, data['y_train_sampled'] = oversampler.fit_resample(X_train, y_train)
    print("Scaling check")
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_sampled)
    data['X_train_scaled_1'] = X_train_scaled[:247].reshape((1, -1))
    data['X_train_scaled_2'] = X_train_scaled[247:].reshape((1, -1))
    data['X_test_scaled'] = scaler.transform(X_test)
    
    print("Returning check")


Using TensorFlow backend.


CHECKPOINT1
CHECKPOINT2
CHECKPOINT3


/home/rz296/miniconda3/envs/partII/lib/python3.6/multiprocessing/popen_fork.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,25

(436, 747668)
(436,)
StratifiedSampling check
Oversampling check


KeyError: 'y_train_sampled'

In [ ]:
manager = Manager()
data = manager.dict()

print("CHECKPOINT1")
p = Process(target=set_Data, args=(data,))
print("CHECKPOINT2")
p.start()
print("CHECKPOINT3")
p.join()

y_train_sampled = data['y_train_sampled']
y_test = data['y_test']
X_train_scaled = np.append(data['X_train_scaled_1'], data['X_train_scaled_2']).reshape(494, 747668)
X_test_scaled = data['X_test_scaled']

# print(y_train_sampled)
# print(X_train_scaled)
print ("Shape of final train and test sets:", X_train_scaled.shape, X_test_scaled.shape)

### Start Methods Here

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from umap.umap_ import UMAP
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
import evalfw

### DR+LR: PPMI

In [ ]:
print(X_train_scaled.shape)
print(y_train_sampled.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
### PCA para: 50
# pca_lr = Pipeline([
#     ("pca", PCA(n_components=250)),
#     ("lr", LogisticRegression(C=100, penalty='l2', max_iter=1000, solver='saga'))
# ])



### UMAP para: 50
umap_lr = Pipeline([
    ("umap", UMAP(n_components=25)),
    ("lr", LogisticRegression(C=100, penalty='l2', max_iter=1000, solver='saga', tol=0.1))
])

# ### ICA para:50
# ica_lr = Pipeline([
#     ("ica", FastICA(n_components=50)),
#     ("lr", LogisticRegression(max_iter=500, penalty='none'))
# ])

# pca_lr.fit(X_train_scaled, y_train_sampled)
umap_lr.fit(X_train_scaled, y_train_sampled)
# ica_lr.fit(X_train_scaled, y_train)

clf_dict = {
#     'PCA and Logistic Regression':pca_lr,
    'UMAP and Logistic Regression': umap_lr,
#     'ICA and Logistic Regression': ica_lr
}

evalfw.eval(clf_dict, X_test_scaled, y_test)

# print("Accuracy of PCA + LR on PPMI testing set: ", accuracy_score(y_test, y_pred_pca_lr))
# print("Accuracy of UMAP + LR on PPMI testing set: ", accuracy_score(y_test, y_pred_umap_lr))
# print("Accuracy of ICA + LR on PPMI testing set: ", accuracy_score(y_test, y_pred_ica_lr))

#### C = 0.01, pca_n=40
Shape of X and y: (88, 40) (88,)  
Current model:  PCA and Logistic Regression  
Confusion matrix of  PCA and Logistic Regression is:  [[11 15]
 [19 43]]  
Accuracy of  PCA and Logistic Regression is:  0.6136363636363636  
Precision of  PCA and Logistic Regression is:  0.7413793103448276  
Recall of  PCA and Logistic Regression is:  0.6935483870967742  
F1 of  PCA and Logistic Regression is:  0.7166666666666668  



##### C = 0.1 pca_n = 40  
Shape of X and y: (88, 40) (88,)  
Current model:  PCA and Logistic Regression  
Confusion matrix of  PCA and Logistic Regression is:  [[11 15]
 [22 40]]  
Accuracy of  PCA and Logistic Regression is:  0.5795454545454546  
Precision of  PCA and Logistic Regression is:  0.7272727272727273  
Recall of  PCA and Logistic Regression is:  0.6451612903225806  
F1 of  PCA and Logistic Regression is:  0.6837606837606838  

##### C = 100 pca_n = 40 
Shape of X and y: (88, 40) (88,)  
Current model:  PCA and Logistic Regression  
Confusion matrix of  PCA and Logistic Regression is:  [[14 12]
 [18 44]]  
Accuracy of  PCA and Logistic Regression is:  0.6590909090909091  
Precision of  PCA and Logistic Regression is:  0.7857142857142857  
Recall of  PCA and Logistic Regression is:  0.7096774193548387  
F1 of  PCA and Logistic Regression is:  0.7457627118644068  


### DR+SVM


In [ ]:
### PCA para: 200
pca_svm = Pipeline([
    ("pca", PCA(n_components=250)),
    ("lr", SVC(C=1000000, kernel='rbf'))
])

### UMAP para: 100
umap_svm = Pipeline([
    ("umap", UMAP(n_components=100)),
    ("lr", SVC(C=1000000, kernel='rbf'))
])

### ICA para:250
ica_svm = Pipeline([
    ("ica", FastICA(n_components=250)),
    ("lr", SVC(C=1000000, kernel='rbf'))
])

pca_svm.fit(X_train_scaled, y_train)
umap_svm.fit(X_train_scaled, y_train)
ica_svm.fit(X_train_scaled, y_train)

clf_dict = {
    'PCA and SVM':pca_svm,
    'UMAP and SVM': umap_svm,
    'ICA and SVM': ica_svm
}

evalfw.eval(clf_dict, X_test_scaled, y_test)

# print("Accuracy of PCA + SVM: ", accuracy_score(y_test, y_pred_pca_svm))
# print("Accuracy of UMAP + SVM: ", accuracy_score(y_test, y_pred_umap_svm))
# print("Accuracy of ICA + SVM: ", accuracy_score(y_test, y_pred_ica_svm))

### Regularisation+LR